In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/open

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

In [ ]:
# RLE 인코딩 함수
# RLE(Run-Length Encoding)은 연속적으로 반복되는 값을 압축하는 방법 중 하나입니다.

def rle_encode(mask):
    # 입력된 이진 마스크를 1차원 배열로 변환합니다.
    pixels = mask.flatten()

    # 배열의 시작과 끝에 0을 추가하여 RLE 알고리즘을 적용하기 위한 기본 준비를 합니다.
    pixels = np.concatenate([[0], pixels, [0]])

    # 연속하지 않는 값의 인덱스를 찾아내어 압축합니다.
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1

    # 압축된 런 길이를 계산합니다.
    runs[1::2] -= runs[::2]

    # 압축된 런을 문자열로 변환하여 반환합니다.
    return ' '.join(str(x) for x in runs)

In [ ]:
# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        # 주어진 CSV 파일을 읽어 데이터를 로드합니다.
        self.data = pd.read_csv(csv_file)

        # 데이터 변환(transform) 및 추론 모드 여부(infer)를 설정합니다.
        self.transform = transform
        self.infer = infer

    def __len__(self):
        # 데이터셋의 길이를 반환합니다.
        return len(self.data)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)를 사용하여 이미지 및 마스크 파일 경로를 가져옵니다.
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            # 추론 모드(infer)인 경우, 이미지 변환(transform)을 적용하고 이미지만 반환합니다.
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_path = self.data.iloc[idx, 2]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # 마스크 이미지에서 픽셀 값이 255인 부분을 12로 간주하여 변경합니다.
        mask[mask == 255] = 12

        if self.transform:
            # 훈련 또는 검증 모드인 경우, 이미지와 마스크에 변환을 적용합니다.
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # 이미지와 마스크를 반환합니다.
        return image, mask

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
# 데이터 변환 파이프라인을 설정합니다. 이 파이프라인은 이미지에 대한 전처리 작업을 정의합니다.
#albumentations를 A로 두고 transform사용 -> albumentations는 torchvision보다 더 다양하고 빠른 모델임..
transform = A.Compose(
    [
        # 이미지를 224x224 크기로 조정합니다.
        A.Resize(224, 224),

        # 이미지를 정규화합니다. (평균과 표준편차를 사용하여 스케일링)
        A.Normalize(),

        # 이미지를 PyTorch 텐서로 변환합니다.
        ToTensorV2()
    ]
)


In [ ]:
train_dataset = CustomDataset(csv_file='train_source.csv', transform=transform)


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
# 이미지를 읽어옵니다.
example = cv2.imread("/content/drive/MyDrive/open/train_source_image/TRAIN_SOURCE_0000.png")

# OpenCV는 이미지를 BGR 형식으로 읽어오기 때문에 RGB로 변환해줍니다.
example = cv2.cvtColor(example, cv2.COLOR_BGR2RGB)

# 이미지를 출력합니다.
plt.imshow(example)
plt.axis('off')  # 축을 숨깁니다.
plt.show()

In [ ]:
segmentation_map = cv2.imread ("/content/drive/MyDrive/open/train_source_gt/TRAIN_SOURCE_0000.png")


segmentation_map

In [ ]:
np.unique(segmentation_map)

In [ ]:
def color_palette():
    """Color palette that maps each class to RGB values.

    This one is actually taken from ADE20k.
    """
    return [
    [0, 0, 0],     # 검정
    [255, 255, 255], # 흰색
    [255, 0, 0],   # 빨강
    [0, 255, 0],   # 초록
    [0, 0, 255],   # 파랑
    [255, 255, 0], # 노랑
    [255, 0, 255], # 마젠타
    [0, 255, 255], # 시안
    [128, 128, 128], # 회색
    [128, 0, 0],   # 진한 빨강
    [0, 128, 0],   # 진한 초록
    [0, 0, 128],   # 진한 파랑
    ]

palette = color_palette()

In [ ]:
# 이미지를 2D 형태로 변환
segmentation_map = cv2.imread("/content/drive/MyDrive/open/train_source_gt/TRAIN_SOURCE_0000.png", cv2.IMREAD_GRAYSCALE)

# 또는 다음과 같이 할 수도 있습니다.
# segmentation_map = cv2.imread("/content/drive/MyDrive/open/train_source_gt/TRAIN_SOURCE_0000.png")
# segmentation_map = cv2.cvtColor(segmentation_map, cv2.COLOR_BGR2GRAY)

segmentation_map = np.array(segmentation_map)

In [ ]:
segmentation_map

In [ ]:
labels = {
    0: [6,7],
    1: [8],
    2: [11,12,13,15],
    3: [17,18,14],
    4: [1],
    5: [19],
    6: [20],
    7: [21],
    8: [23],
    9: [24,25],
    10: [],
    11: [26,27,28,29,30,32,33]
}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
image = example
color_segmentation_map = np.zeros((segmentation_map.shape[0], segmentation_map.shape[1], 3), dtype=np.uint8) # height, width, 3
for label, color in enumerate(palette):
    color_segmentation_map[segmentation_map - 1 == label, :] = color
# Convert to BGR
ground_truth_color_seg = color_segmentation_map[..., ::-1]

img = np.array(image) * 0.5 + ground_truth_color_seg * 0.5
img = img.astype(np.uint8)

plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.show()

In [ ]:
train_ds = {
    'train': {
        'pixel_values': image,  # 여기서 my_image는 단일 이미지 데이터입니다.
        'label': segmentation_map,  # 여기서 my_label은 해당 이미지의 라벨입니다.
    },
    'num_rows': 1  # 단일 이미지이므로 num_rows는 1입니다.
}
test_image = cv2.imread("/content/drive/MyDrive/open/test_image/TEST_0000.png")

test_ds = {
    'test': {
        'pixel_values': image,  # 여기서 my_image는 단일 이미지 데이터입니다.
        'label': segmentation_map,  # 여기서 my_label은 해당 이미지의 라벨입니다.
    },
    'num_rows': 1  # 단일 이미지이므로 num_rows는 1입니다.
}

In [ ]:
import numpy as np
from torch.utils.data import Dataset

class ImageSegmentationDataset(Dataset):
    """Image segmentation dataset."""

    def __init__(self, dataset, transform):
        """
        Args:
            dataset
        """
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        original_image = np.array(self.dataset[idx]['pixel_values'])
        original_segmentation_map = np.array(self.dataset[idx]['label'])

        transformed = self.transform(image=original_image, mask=original_segmentation_map)
        image, segmentation_map = transformed['image'], transformed['mask']

        # convert to C, H, W
        image = image.transpose(2,0,1)

        return image, segmentation_map, original_image, original_segmentation_map

In [ ]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

train_transform = A.Compose([
    A.LongestMaxSize(max_size=1333),
    A.RandomCrop(width=512, height=512),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

test_transform = A.Compose([
    A.Resize(width=512, height=512),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),

])

train_dataset = ImageSegmentationDataset(train_ds, transform=train_transform)
# test_dataset = ImageSegmentationDataset(test_ds, transform=test_transform)

In [ ]:
len(train_dataset)


In [ ]:
image, segmentation_map, _, _ = train_dataset[0]
print(image.shape)
print(segmentation_map.shape)